# Assignment 2
# Ling Fei Zhang, 260985358
# Brandon Ma, ID

## Question 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from functions import SARSA, visualize
import time
import gymnasium as gym


c:\Users\Ling0\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here, we will set a few parameters for our environment.

In [2]:
ALPHAS = [0.01, 0.1, 0.2]
GAMMA = 0.9
TEMPERATURE = [1, 10, 100]
EPISODES = 5500
SEEDS = np.arange(10)


Driver Code to test out the combination of each alpha with each temperature for SARSA.

In [3]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)
env.reset()
sarsa = SARSA(env, 0.1, 1, GAMMA, EPISODES, False)
rewards = sarsa.simulate_episodes()
final_policy = sarsa.learned_policy
print("final policy:\n", final_policy.reshape(4,4)) #tabular format to better visualize
# print("total reward: ", rewards)

#uncomment this line if you want to vizualize the game
# visualize(final_policy, 10)

 17%|█▋        | 931/5500 [00:00<00:00, 4670.70it/s]

Simulating episode 0.
Simulating episode 1.
Simulating episode 2.
Simulating episode 3.
Simulating episode 4.
Simulating episode 5.
Simulating episode 6.
Simulating episode 7.
Simulating episode 8.
Simulating episode 9.
Simulating episode 10.
Simulating episode 11.
Simulating episode 12.
Simulating episode 13.
Simulating episode 14.
Simulating episode 15.
Simulating episode 16.
Simulating episode 17.
Simulating episode 18.
Simulating episode 19.
Simulating episode 20.
Simulating episode 21.
Simulating episode 22.
Simulating episode 23.
Simulating episode 24.
Simulating episode 25.
Simulating episode 26.
Simulating episode 27.
Simulating episode 28.
Simulating episode 29.
Simulating episode 30.
Simulating episode 31.
Simulating episode 32.
Simulating episode 33.
Simulating episode 34.
Simulating episode 35.
Simulating episode 36.
Simulating episode 37.
Simulating episode 38.
Simulating episode 39.
Simulating episode 40.
Simulating episode 41.
Simulating episode 42.
Simulating episode 43

 34%|███▎      | 1853/5500 [00:00<00:00, 4330.88it/s]

Simulating episode 931.
Simulating episode 932.
Simulating episode 933.
Simulating episode 934.
Simulating episode 935.
Simulating episode 936.
Simulating episode 937.
Simulating episode 938.
Simulating episode 939.
Simulating episode 940.
Simulating episode 941.
Simulating episode 942.
Simulating episode 943.
Simulating episode 944.
Simulating episode 945.
Simulating episode 946.
Simulating episode 947.
Simulating episode 948.
Simulating episode 949.
Simulating episode 950.
Simulating episode 951.
Simulating episode 952.
Simulating episode 953.
Simulating episode 954.
Simulating episode 955.
Simulating episode 956.
Simulating episode 957.
Simulating episode 958.
Simulating episode 959.
Simulating episode 960.
Simulating episode 961.
Simulating episode 962.
Simulating episode 963.
Simulating episode 964.
Simulating episode 965.
Simulating episode 966.
Simulating episode 967.
Simulating episode 968.
Simulating episode 969.
Simulating episode 970.
Simulating episode 971.
Simulating episo

 58%|█████▊    | 3189/5500 [00:00<00:00, 4372.53it/s]

Simulating episode 1853.
Simulating episode 1854.
Simulating episode 1855.
Simulating episode 1856.
Simulating episode 1857.
Simulating episode 1858.
Simulating episode 1859.
Simulating episode 1860.
Simulating episode 1861.
Simulating episode 1862.
Simulating episode 1863.
Simulating episode 1864.
Simulating episode 1865.
Simulating episode 1866.
Simulating episode 1867.
Simulating episode 1868.
Simulating episode 1869.
Simulating episode 1870.
Simulating episode 1871.
Simulating episode 1872.
Simulating episode 1873.
Simulating episode 1874.
Simulating episode 1875.
Simulating episode 1876.
Simulating episode 1877.
Simulating episode 1878.
Simulating episode 1879.
Simulating episode 1880.
Simulating episode 1881.
Simulating episode 1882.
Simulating episode 1883.
Simulating episode 1884.
Simulating episode 1885.
Simulating episode 1886.
Simulating episode 1887.
Simulating episode 1888.
Simulating episode 1889.
Simulating episode 1890.
Simulating episode 1891.
Simulating episode 1892.


 82%|████████▏ | 4494/5500 [00:01<00:00, 4219.12it/s]

Simulating episode 3189.
Simulating episode 3190.
Simulating episode 3191.
Simulating episode 3192.
Simulating episode 3193.
Simulating episode 3194.
Simulating episode 3195.
Simulating episode 3196.
Simulating episode 3197.
Simulating episode 3198.
Simulating episode 3199.
Simulating episode 3200.
Simulating episode 3201.
Simulating episode 3202.
Simulating episode 3203.
Simulating episode 3204.
Simulating episode 3205.
Simulating episode 3206.
Simulating episode 3207.
Simulating episode 3208.
Simulating episode 3209.
Simulating episode 3210.
Simulating episode 3211.
Simulating episode 3212.
Simulating episode 3213.
Simulating episode 3214.
Simulating episode 3215.
Simulating episode 3216.
Simulating episode 3217.
Simulating episode 3218.
Simulating episode 3219.
Simulating episode 3220.
Simulating episode 3221.
Simulating episode 3222.
Simulating episode 3223.
Simulating episode 3224.
Simulating episode 3225.
Simulating episode 3226.
Simulating episode 3227.
Simulating episode 3228.


100%|██████████| 5500/5500 [00:01<00:00, 4298.47it/s]

Simulating episode 4494.
Simulating episode 4495.
Simulating episode 4496.
Simulating episode 4497.
Simulating episode 4498.
Simulating episode 4499.
Simulating episode 4500.
Simulating episode 4501.
Simulating episode 4502.
Simulating episode 4503.
Simulating episode 4504.
Simulating episode 4505.
Simulating episode 4506.
Simulating episode 4507.
Simulating episode 4508.
Simulating episode 4509.
Simulating episode 4510.
Simulating episode 4511.
Simulating episode 4512.
Simulating episode 4513.
Simulating episode 4514.
Simulating episode 4515.
Simulating episode 4516.
Simulating episode 4517.
Simulating episode 4518.
Simulating episode 4519.
Simulating episode 4520.
Simulating episode 4521.
Simulating episode 4522.
Simulating episode 4523.
Simulating episode 4524.
Simulating episode 4525.
Simulating episode 4526.
Simulating episode 4527.
Simulating episode 4528.
Simulating episode 4529.
Simulating episode 4530.
Simulating episode 4531.
Simulating episode 4532.
Simulating episode 4533.


In [22]:
for alpha in ALPHAS:
    for temp in TEMPERATURE:
        env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)
        env.reset()
        sarsa = SARSA(env, alpha, temp, GAMMA, EPISODES, False)

### Effect of parameters on final training performance for SARSA

### Effect of parameters on final testing performance for SARSA

### Best parameter for SARSA

### Conclusion for SARSA

### Effect of parameters on final training performance for EXPECTED SARSA

### Effect of parameters on final testing performance for EXPECTED SARSA

### Best parameter for EXPECTED SARSA

### Conclusion for EXPECTED SARSA